# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [839]:
import pandas as pd
import numpy as np
import glob
DATA_FOLDER = 'Data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

## Task 1: Report 
The first step was to import all the data from the ebola data floder. We created a different list of path for each country and then concatenated all the fill to create one dataframe per country.
The second step was to look at the data to find relevant columns, value, etc... to create a Dataframe that will contains the more informative data.

The first thing that we noticed is that real-life data is messy! Each country used different labels, format, value or even how they treat a missing value. Moreover it is easy to see that the data contains "mistakes" that can totally change our final summary data. One stiking example resides in the Liberia data, where the cumulative value of cases in december are written in the new cases rows. Missing that incoherence would lead to totally wrong statistics.

Based on those observations we did choices on what value we were going to use.
We decided to chose values that looked "stables". In the sens where we choosed values that we evaluate as realistic and that minimizes outling data.

Our choices are the following:

    -For the Guinea data: The data of guinea was quite clean. We chose the variable "New deaths registered" for the deaths and "new cases of confirmed" for the new cases.
    This choice was based on the fact that for the deaths the data had no suspect value and that the field was present on all daily report. Concerning the new cases, we've made the choice of taking only the confirmed cases because the field Total cases was totally incoherent. The values where not always increasing the next day. Even if this is possible i.e suspected cases becomes healthy, it didn't look very acurate. Since both data for cases and death were daily data we just had to aggregate by month using a mean.
    
    -For the Sierra Leone data: We chose the "new confirmed" and the "death confirmed" variables. For pretty much the same reasons as above. It looked to us that it was the most relevant and consistant data. 
    The main difference is that in this case the cases value are daily values and the deaths value are cumulative values. Therefore we worked with two different dataframe in order to aggregate the first with a mean fonction and the second with a max fonction, then minus the precedent value (last month), then divided by 30 in order to have a daily average. We will discuss the efficiency of this method in the results discussion.
    
    -For the Liberia data: This set of data was the worse. Incoherence as exposed above (with the wrong row), new fields added, other deleted, or some other juste not filled anymore. In order to have, in our opinion, the best data we decided to use two different "set of values" depending on the date. For daily report until 2014-10-08 we sumed the fields "Total cases suspected", "Total cases probable" and "Total cases confirmed". For the following dates we used the fields "Cumulative ... cases". This choice was mainly motivated by the error present from the daily report of 2014-12-04, where the rows are wrongly filled. And since the cumulative field is not present in the early daily reports it was either drop the data from the 4 december or used two different strategies and then merge. We decided to merge.


## Task 1: Code 

In [840]:
#Test and vizualise the shape of the data
from IPython.display import display
from datetime import date
guinea = DATA_FOLDER+'/ebola/guinea_data/*.csv'
liberia = DATA_FOLDER+'/ebola/liberia_data/*.csv'
sierra = DATA_FOLDER+'/ebola/sl_data/*.csv'

Parses the dates values in order to keep only the Y-M pattern

In [841]:
def parse_date(data):
    return data.Date.map(lambda x: '-'.join(str(x).split('-')[:2]))

Read all the files in the 'country' folder and concanate them in single dataframe for the country

In [842]:
def country_dataframe(country):
    df = pd.read_csv(glob.glob(country)[0])
    for file in glob.glob(country)[1:]:
        df = pd.concat([df,pd.read_csv(file)],axis = 0)
    return df

Compute the daily average per month for Dataframe that used cumulative data

In [843]:
def compute_day_av(df):
    lastmonth_death = 0
    for value in df.index:
        curr = df.loc[value].copy()
        new_val = curr- lastmonth_death
        lastmonth_death = curr 
        df.loc[value] = new_val/30.0
    return 0

Helper function to weight the month cumulative data that had reports only until the ~10th

In [844]:
def correct_cumulative_av(df,index):
    df.loc[index] = df.loc[index].copy()*3
    return 0

This function does most of the cleaning.<br>
-It first transform the date fielf of the df as a Date for consistence with the other data<br>
-Then it adds a Country column with the parameter 'country'<br>
-It exctract only the rows of which the Variable value is in the 'isin_list' parameter<br>
-It rename all the variable to unified the data if given as parameter<br>
-It keep only the year and month of the date<br>
-It set the index and keeps only the secified colums<br>
-It drops all NaN values<br>
-It casts all the values to int64 to avoid possible mistakes<br>
-Then groupby and aggregate by the action given in the parameters.<br>
-Sets the only columns to be 'Daily average' to facilitate the final merge<br>

In [845]:
def data_wrangler(df,country, isin_list, v_name, col, func):
    df.Date = pd.to_datetime(df.Date)
    df['Country'] = country
    df = df[df['Variable'].isin(isin_list)].copy()
    if v_name is not None:
        df.Variable = v_name
    df.Date = parse_date(df)
    df = df.set_index(['Country', 'Date', 'Variable'])[col]
    df = pd.DataFrame(df).dropna(axis = 0)
    df = df.astype('int64')
    if func == 'max':
        df = df.groupby(level=[0,1,2]).max()
    elif func == 'mean':
        df = df.groupby(level=[0,1,2]).mean()
    df.columns = ['Daily Average']
    return df

Sierra Leone

In [ ]:
s = country_dataframe(sierra)
#rename the colums to be consistant with the other countries
s = s.rename(index=str,columns={'date':'Date','variable':'Variable'})
#new cases for SL
s_cases = data_wrangler(s,'Sierra-Leone', ['new_confirmed'], 'New cases confirmed','National', 'mean')
#death for Sierra Leone
s_death = data_wrangler(s, 'Sierra-Leone', ['death_confirmed'], 'New deaths confirmed', 'National', 'max')
compute_day_av(s_death)
#merges cases and deaths
s = pd.merge(s_cases.reset_index(), s_death.reset_index(), how= 'outer')
s = s.set_index(['Country','Date','Variable'])

Guinea

In [ ]:
g = country_dataframe(guinea)
#rename the colums to be consistant with the other countries
g = g.rename(columns={'Description':'Variable'})
#cases and death for Guinea
g = data_wrangler(g, 'Guinea', ['New cases of confirmed', 'New deaths registered'], None , 'Totals', 'mean')

Liberia

In [ ]:
l = country_dataframe(liberia)
#death for Liberia
l_death = data_wrangler(l, 'Liberia', ['Newly reported deaths'], None, 'National', 'mean')
#cases until the 2014-10-8
l_cases_1 = l[l['Variable'].isin(['Cumulative confirmed, probable and suspected cases',
                               'Cumulative (confirmed + probable + suspects)'])].copy()
l_cases_1.Variable = 'New cases(confirmed, probable, suspecet)'
#cases from the 2014-10-08
l_cases_2 = l[l['Variable'].isin(['Total suspected cases','Total probable cases',
                               'Total confirmed cases'])].copy()
l_cases_2 = l_cases_2[l_cases_2.Date < date(2014, 10, 8) ]
l_cases_2.Variable = 'New cases(confirmed, probable, suspeced)'
l_cases_2 = l_cases_2.groupby(['Country', 'Date', 'Variable']).sum()
#merges cases
l_cases = pd.merge(l_cases_1.reset_index(), l_cases_2.reset_index(), how= 'outer')
l_cases.Date = parse_date(l_cases)
l_cases = l_cases.set_index(['Country','Date', 'Variable'])['National'].sort_index(level=1)
l_cases = l_cases.groupby(level=[0,1,2]).max()
l_cases = pd.DataFrame(l_cases)
compute_day_av(l_cases)
l_cases.columns = ['Daily Average']
#merges the cases and the deaths
l = pd.merge(l_death.reset_index(), l_cases.reset_index(), how= 'outer')
l = l.set_index(['Country','Date','Variable'])

Merge the three countries dataframes

In [846]:
m = pd.merge(s.reset_index(),(pd.merge(g.reset_index(),l.reset_index(),how= 'outer')),how='outer')
m = m.set_index(['Country', 'Date', 'Variable'])
m=m.sort_index(level=[0,1])
#Weights the december values
i1 = ('Liberia', '2014-12', 'New cases(confirmed, probable, suspecet)')
i2 = ('Sierra-Leone', '2014-12', 'New deaths confirmed')
correct_cumulative_av(m, i1)
correct_cumulative_av(m, i2)
m

Daily Average
Country      Date    Variable                                               
Guinea       2014-08 New cases of confirmed                        12.400000
                     New deaths registered                          3.750000
             2014-09 New cases of confirmed                        13.000000
                     New deaths registered                          3.562500
             2014-10 New cases of confirmed                         6.000000
                     New deaths registered                         15.000000
Liberia      2014-06 New cases(confirmed, probable, suspeced)       3.000000
                     Newly reported deaths                          2.000000
             2014-07 New cases(confirmed, probable, suspeced)       7.233333
                     Newly reported deaths                          4.272727
             2014-08 New cases(confirmed, probable, suspeced)      42.500000
                     Newly reported deaths                         23.222222
             2014-09 New cases(confirmed, probable, suspeced)      74.800000
                     Newly reported deaths                         37.608696
             2014-10 New cases(confirmed, probable, suspeced)       8.066667
                     New cases(confirmed, probable, suspecet)      81.900000
                     Newly reported deaths                         28.040000
             2014-11 New cases(confirmed, probable, suspecet)      37.600000
                     Newly reported deaths                         15.538462
             2014-12 New cases(confirmed, probable, suspecet)      14.400000
Sierra-Leone 2014-08 New cases confirmed                           20.611111
                     New deaths confirmed                          12.900000
             2014-09 New cases confirmed                           36.962963
                     New deaths confirmed                           5.433333
             2014-10 New cases confirmed                           56.714286
                     New deaths confirmed                          17.333333
             2014-11 New cases confirmed                           66.210526
                     New deaths confirmed                          14.200000
             2014-12 New cases confirmed                           54.333333
                     New deaths confirmed                          21.200000

## Task 1: Results discussion 

At first glance we are quite happy with the results. Our computation doesn't lead to any negative values or NaN or any other absurde values. All the daily averages seems to be in the same range.<br>
However we can still makes some comments.<br>
First of all, some data take the value of confirmed, probable and suspected cases/deaths and some others take only the confirmed cases/deaths. We could argue against this choice but we thought that is was the best way or the most informativ way of representing the data, regarding the data sets.<br>
The seconds comments that we could do is that daily values are better than cumulative values. With daily values we can aggregate them using a mean which is quite robust, its is not too dependant on the missing values or the report dates. On the other side cumulative value are dependant to those things especially on the dates. Imagine the case where for one month only one report from the 3rd day of this month is available. We would have took the value of the third day as the max value from this month then we would have substracted the max value of the last month and then weighted it by the number of days in one month. Which would have lead to a to small value to be representative. However in our dataset most month data that used cumulative values had report for advanced day of the month. Except for the month 12 for the "New deaths confirmed" of the Sierra-Leone data and the "New cases(confirmed, probable, suspecet)" also of the month 12 of the Liberia data, where the reports stop around the tenth. Therefore for those 2 cases, we multiplied the values by 3 since it's a daily average.<br>
The last remark will be that for the month of december in the Liberia data we can see that there is no value of the "Newly reported deaths". It's is juste because this field had no data for this month.


## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [847]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [848]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

Name,Labels,Units,Levels,Storage,NAs
pclass,,,3,integer,0
survived,Survived,,,double,0
name,Name,,,character,0
sex,,,2,integer,0
age,Age,Year,,double,263
sibsp,Number of Siblings/Spouses Aboard,,,double,0
parch,Number of Parents/Children Aboard,,,double,0
ticket,Ticket Number,,,character,0
fare,Passenger Fare,British Pound (\243),,double,1
cabin,,,187,integer,0


For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [849]:
# Write your answer here